In [ ]:
import numpy as np
import itertools

In [ ]:
# radix vector
R = np.array ([7,24,60,60,1])

In [ ]:
# weight vector
W = np.cumproduct( R[::-1]) [::-1]
np.vstack ((R,W))

array([[     7,     24,     60,     60,      1],
       [604800,  86400,   3600,     60,      1]])

In [ ]:
def dhms2s (DHMS):
    """(days, hours, minutes, seconds) to seconds.
    
    Implemented as a dot product with right part of W
    
    DHMS : n x 4 matrix of n DHMS tuples
    out  : n x 1 matrix of n seconds  
    """
    return (DHMS @ W[1:]).reshape (-1,1)


def s2dhms (s):
    """Seconds tp (days, hours, minutes, seconds).
    
    Implemented as
        - modulo by left part of W
        - followed by integral division by right part of W
    
    s    : n x 1 matrix of n seconds 
    out  : n x 4 matrix of n DHMS tuples
    """

    return s % W[:-1] // W [1:] 

In [ ]:
DHMS = np.zeros ((W[0],4), dtype = np.uint32)
S    = np.arange (W[0], dtype=np.uint32).reshape (W[0],1)

i = 0
for d in range (R[0]):
    for h in range (R[1]):
        for m in range (R[2]):
            for s in range (R[3]):

                DHMS [i] = d,h,m,s
                i += 1

In [ ]:
DHMS

array([[ 0,  0,  0,  0],
       [ 0,  0,  0,  1],
       [ 0,  0,  0,  2],
       ...,
       [ 6, 23, 59, 57],
       [ 6, 23, 59, 58],
       [ 6, 23, 59, 59]], dtype=uint32)

In [ ]:
S

array([[     0],
       [     1],
       [     2],
       ...,
       [604797],
       [604798],
       [604799]], dtype=uint32)

In [ ]:
(dhms2s (DHMS) == S).all()

True

In [ ]:
(s2dhms (S) == DHMS).all()

True

In [ ]:
## nD 

n = 3
R = np.array ([5,2,4,2,3,2,2,2,1,2,1]) [9-2*n+1:]
# R = np.array ([    4,2,3,2,2,2,1,2,1])

W = np.cumproduct( R[::-1]) [::-1]
np.vstack ((R,W))

array([[ 3,  2,  2,  2,  1,  2,  1],
       [48, 16,  8,  4,  2,  2,  1]])

In [ ]:
d = 83
d_radix = s2dhms (d)
d_radix
# dhms2s (d_radix)

array([2, 0, 0, 1, 0, 1])

    0123 [0] = 0
     123 [2] = 3
     12  [0] = 1
      2  [0] = 2

In [ ]:
def radix2axes (d_radix):
    result = d_radix.copy()
    axes = [i for i in np.arange (n)]
    for i in range (0,len (d_radix),2):
        j = d_radix[i]
        ax = axes [j]
        result [i] = ax
        axes.remove (ax)
    return result

In [ ]:
def axes2radix (d_axes):
    result = d_axes.copy()
    axes = [i for i in np.arange (n)]

    for i in range (0, len (d_axes), 2):
        j = d_axes [i]
        rad = list (axes).index (j)
        result [i] = rad
        axes.remove (j)
    return result

In [ ]:
d_radix [::2]

array([2, 0, 0])

In [ ]:
axes2radix (_)

array([2, 0, 0])

In [ ]:
radix2axes (_)

array([2, 0, 0])

## nD 

In [ ]:
dimensions = [1,1,1,1,1,1,1] # shape of the voxel scene 
n = len (dimensions) # dimensionality

R = np.array ([8,2, 7,2, 6,2, 5,2, 4,2, 3,2, 2,2, 1,2, 1]) [16-2*n:]    # e.g, for n = 3: [3,2, 2,2, 1,2, 1]
W = np.cumproduct( R[::-1]) [::-1]


nDarts = W[0] # darts of one n-cell
assert nDarts == 2**n * np.product (np.arange (1,1+n)) == np.product (np.arange(2*n,1,-2))  # 2^n * n! == (2n)!!


alpha_star = np.zeros ((nDarts,n+1), dtype=np.uint64)
delta      = np.zeros ((nDarts,n  ), dtype=np. int64)

for d in range (nDarts):
    # alpha_star[0]
    d_radix = s2dhms (d)
    d_radix [-1] = 1 - d_radix[-1]
    alpha_star[d,0] = dhms2s (d_radix)[0,0]

    # alpha_star[n]
    d_radix = s2dhms (d)
    d_radix [1] = 1 - d_radix[1]
    alpha_star[d,n] = dhms2s (d_radix)[0,0]
    
    # alpha_star[i] for 1<i<n
    for i in range (1,n):
        d_axes = radix2axes (s2dhms (d))
        # swapping 2-tuples
        a,b,c,e = d_axes[2*n-2*(i+1):2*n-2*(i+1)+4]
        d_axes[2*n-2*(i+1):2*n-2*(i+1)+4] = c,e,a,b
        alpha_star[d,i] = dhms2s (axes2radix (d_axes))[0,0]

    # delta
    xn1,in1 = radix2axes (s2dhms (d))[:2]
    delta [d] = [-1,1][in1] * np.eye(n,dtype=int)[xn1]

#     print table
#     print (f'{d:2}', end=': ')
#     for a in alpha_star[d]:
#         print (f'{a:3}', end=' ')
#     print (end = '   [')
#     for de in delta[d]:
#         print (f'{de:2}', end= ' ')
#     print (']')

In [ ]:
# export alpha star

# header
header  = [f'a{i}' for i in range (n+1)]
header += [f'd{i}' for i in range (n)]
header  = '\t'.join (header)


TABLE = np.hstack ((alpha_star, delta)).astype (int)
np.savetxt (
    f'LUT/LUT_{n}D_alpha_star.tsv',
    TABLE,
    delimiter='\t',
    fmt='%4d',
    header = header
)

np.save (
    f'LUT/LUT_{n}D_alpha_star',
    TABLE
)

## Proofs of concept

In [ ]:
from combinatorial.gmaps import nGmap

In [ ]:
# single n-cell

alpha = alpha_star.copy()
alpha[:,-1] = np.arange (nDarts) # boundary: alpha_n = Id
G = nGmap (alpha.T)
G

7-gMap of 645120 darts:
  # 0-cells: 128
  # 1-cells: 448
  # 2-cells: 672
  # 3-cells: 560
  # 4-cells: 280
  # 5-cells: 84
  # 6-cells: 14
  # 7-cells: 1
  # ccs    : 1

In [ ]:
# some sanity checks
assert G.no_i_cells(  0) == 2**n
assert G.no_i_cells(n-1) ==  2*n
assert G.no_i_cells(  n) ==    1

# number of all cells in n-hypercube == 3^n
assert (sum (G.no_i_cells (i) for i in range (1+n)) == 3**n)

In [ ]:
# iterate over all positions
# for p in itertools.product( *(range (m) for m in dimensions) ):
#     print (p)

In [ ]:
# radix for dart dimensions x darts-in-n-cell 
RR = dimensions + [W[0],1] # e.fg., [ny,nx,8,1] or [nx,ny,nx,48,1]
WW = np.cumproduct(RR[::-1]) [::-1]
np.vstack ((RR,WW))

array([[     1,      1,      1,      1,      1,      1,      1, 645120,
             1],
       [645120, 645120, 645120, 645120, 645120, 645120, 645120, 645120,
             1]])

In [ ]:
def alpha_i (d,i):
    # dart to (p,s)
    tup = d % WW[:-1] // WW [1:]  
    p,s = tup [:n], tup[-1]
    
    if i == n: # update p
        p += delta [s]
        # borders for alpha_n: if updated p would fall out of scene: return identity
        if np.any (p < 0          ): return d
        if np.any (p >= dimensions): return d

    s = alpha_star [s,i] # update s

    tup [:n],tup[-1] = p,s
    
    return tup @ WW[1:] # tuple x radix

In [ ]:
# construct alpha for n-Gmap given by `dimensions`
alpha = np.zeros ((np.product (dimensions) * nDarts, 1+n), dtype=np.uint64)

for d in range (np.product (dimensions) * nDarts):
    for i in range (1+n):
        alpha [d,i] = alpha_i (d,i) 

In [ ]:
G = nGmap.from_alpha_array (alpha.T)
G

7-gMap of 645120 darts:
  # 0-cells: 128
  # 1-cells: 448
  # 2-cells: 672
  # 3-cells: 560
  # 4-cells: 280
  # 5-cells: 84
  # 6-cells: 14
  # 7-cells: 1
  # ccs    : 1

## Towards membrane-centric endocding


### scenario 2 (internal $90^\circ$ dart flips)

- $\alpha_{n-1}^{90^\circ} (s) = \alpha_{n-1}^* (s)$
- $\Delta^{90^\circ} p (s) = \max \{ 0, \Delta p (\alpha_{n-1}^* (s)) \} - \max \{ 0, \Delta p (s) \}$

In [ ]:
# compute membrane-deltas for interior

alpha_n1_090 = alpha_star[:,n-1]

coords_from  =  delta.copy()
coords_from *= (coords_from != -1) # replace -1s with 0s

coords_to    = delta[alpha_star[:,n-1]].copy()
coords_to   *= (coords_to != -1)   # replace -1s with 0s

# subtract
delta_090 = coords_to - coords_from

In [ ]:
assert (np.maximum (0,delta[alpha_star[:,n-1]]) - np.maximum (0,delta) == delta_090).all()

### scenario 1 (shell, $180^\circ$ dart flips)

- $\alpha_{n-1}^{90^\circ} (s) = \alpha_{n-1}^* ( \alpha_n^* ( \alpha_{n-1}^* (s)))$
- $\Delta^{90^\circ}p(s) = \Delta^{90^\circ}p(s) + \Delta^{90^\circ}p (\alpha_n^* (\alpha_{n-1}^* (s)))$

In [ ]:
alpha_n1_180 = alpha_star [alpha_star [alpha_star[:,n-1], n], n-1]
delta_180    = delta_090 + delta_090 [alpha_star [alpha_star[:,n-1], n]]

### scenario 0 (shell, $270^\circ$ dart flips)

- $\alpha_{n-1}^{270^\circ}(s) = \alpha_n^* (\alpha_{n-1}^* (\alpha_n^* (s)))$
- $\Delta^{270^\circ}p(s) = \Delta^{90^\circ}p( \alpha_n^* (s)))$

In [ ]:
alpha_n1_270 = alpha_star [alpha_star [alpha_star[:,n], n-1], n]
delta_270    = delta_090  [alpha_star[:,n]]

### All three $\alpha_{n-1}$ tables

In [ ]:
# check for permutation
all (np.sort (alpha_n1_270) == np.arange (nDarts)),\
all (np.sort (alpha_n1_180) == np.arange (nDarts)),\
all (np.sort (alpha_n1_090) == np.arange (nDarts))

(True, True, True)

In [ ]:
# check self involutions
all (alpha_n1_270 [alpha_n1_270] == np.arange (nDarts)),\
all (alpha_n1_180 [alpha_n1_180] == np.arange (nDarts)),\
all (alpha_n1_090 [alpha_n1_090] == np.arange (nDarts))

(True, True, True)

In [ ]:
# check self involution deltas
np.all ((delta_270 + delta_270 [alpha_n1_270]) == 0),\
np.all ((delta_180 + delta_180 [alpha_n1_180]) == 0),\
np.all ((delta_090 + delta_090 [alpha_n1_090]) == 0)

(True, True, True)

In [ ]:
# check a0 a2
all (alpha_star [:,0] [alpha_n1_270 [alpha_star[:,0] [alpha_n1_270]]] == np.arange (nDarts)) 

True

In [ ]:
np.vstack ((
    np.arange(nDarts),
    delta_270.T, alpha_n1_270, 
    delta_180.T, alpha_n1_180, 
    delta_090.T, alpha_n1_090, 
)).T.astype (int)

array([[     0,     -1,      0, ...,      0,      0,  92160],
       [     1,     -1,      0, ...,      0,      0,  92161],
       [     2,     -1,      0, ...,      0,      0,  92162],
       ...,
       [645117,      0,      0, ...,      1,     -1, 552957],
       [645118,      0,      0, ...,      1,     -1, 552958],
       [645119,      0,      0, ...,      1,     -1, 552959]])

In [ ]:
# export the table in tab separated values


LUT_DD = np.vstack ((
    delta_270.T, alpha_n1_270, 
    delta_180.T, alpha_n1_180, 
    delta_090.T, alpha_n1_090, 
)).T.astype (np.int32)

# header
header = []
for s in 0,1,2:
    header += [ f's{s}_d{i}' for i in range (n)]
    header += [ f's{s}_a{n-1}']
header = '\t'.join (header)

np.savetxt (
    f'LUT/LUT_{n}D_alpha_{n-1}_scenarios.tsv',
    LUT_DD,
    comments  = '#',
    header    = header,
    fmt       = '%5d',
    delimiter = '\t', 
)

np.save (f'LUT/LUT_{n}D_alpha_{n-1}_scenarios', LUT_DD)

### Scenario decisions

The scenario of a dart is based on

- dimenions $m = (m_0, m_1 ... m_{n-1} )$ of the hyper volume
- position $p = (p_0, p_1 ... p_{n-1} )$
- first 4 dart descriptors $x_{n-1} I_{n-1} x_{n-2} I_{n-2} \ldots$

The logic is as follows:


- 2. scenario: Most of darts are internal, i.e., not belonging to the background cell $n$-cell.

- 1. scenario: To belong to the background $n$-cell, in general: $p_{x_{n-1}}$ must equal either to zero or to the corresponding dimensions: 
    - if $I_{n-1} = 1$ (i.e, ↑): $p_{x_{n-1}} = 0$
    - if $I_{n-1} = 0$ (i.e, ↓): $p_{x_{n-1}} = m_{x_{n-1}}$  
    - The above two test can be combined into the following single test:
    - $p_{x_{n-1}} = m_{x_{n-1}}  (1 - I_{n-1})$
  
- 0. scenario: If already in the background $n$-cell, (i.e., previous test test holds) to qualify additionally for a corner/edge/... $p_{x_n-2}$ must equal either to zero or to the corresponding dimension minus 1:
    - if $I_{n-2} = 1$ (i.e, ↑): $p_{x_{n-2}} = m_{x_{n-2}}-1$
    - if $I_{n-2} = 0$ (i.e, ↓): $p_{x_{n-2}} = 0$
    - The above two test can be combined into the following single test:
    - $p_{x_{n-2}} = I_{n-2} (m_{x_{n-2}}-1)$

In [ ]:
LUT_PosDartDescriptor = np.zeros ((nDarts, 2*n), dtype = np.uint8)
for d in range (nDarts):
    LUT_PosDartDescriptor [d] = radix2axes (s2dhms ([d])) 

# columns 1 and 3 must be either 0 or 1 (down / up arrow)
assert np.all (LUT_PosDartDescriptor [:,1::2] <= 1)
assert np.all (LUT_PosDartDescriptor [:,1::2] >= 0)

assert np.all (LUT_PosDartDescriptor [:,0::2] < n)
assert np.all (LUT_PosDartDescriptor [:,0::2] >= 0)

# header
header = [f'x{i}\tI{i}' for i in range (n-1,-1,-1)]
header = '\t'.join (header)

np.savetxt (
  f'LUT/LUT_{n}D_PosDartDescriptor.tsv',
    LUT_PosDartDescriptor,   
    comments  = '#',
    header    = header,
    fmt       = '%1d',
    delimiter = '\t', 
)

np.save (f'LUT/LUT_{n}D_PosDartDescriptor', LUT_PosDartDescriptor)

In [ ]:
m = dimensions
print (m)
for p in [[0,0,0]]:#,[0,1],[0,2]]:
    print (40*'-')
    for d in range (nDarts):
        # take the first four descriptors
        x_n1, I_n1, x_n2, I_n2 = radix2axes (s2dhms ([d])) [:4]

        # preparing for worst: internal scenario:
        s = 2
        
        if p[x_n1] == m [x_n1] * (1 - I_n1):
#         if I_n1 == 0 and p[x_n1] == m [x_n1] or \
#            I_n1 == 1 and p[x_n1] == 0:
            s = 1

            if p[x_n2] == I_n2 * (m[x_n2]-1):
#             if I_n2 == 0 and p[x_n2] == 0         or \
#                I_n2 == 1 and p[x_n2] == m[x_n2]-1 :
                    s = 0
        print (p, f'{d:2}', s) #, '...', m [r[2]])
